# Travailler avec des données

Ce notebook a pour objectif de vous aider à explorer la base de données BioTime disponible ici : https://zenodo.org/record/5026943
Ces données sont trop lourdes, trop nombreuses, pour travailler avec Excel.

Ouvrons ces données à l'aide de R.

En préalable, il faut charger la librairie "tidyverse", indispensable pour lire et travailler avec un fichier .CSV

In [ ]:
library(tidyverse)

Ouvrons le fichier CSV à l'aide de la fonction "read_csv". Les données se nommeront "data" dans la suite du code. En rouge, c'est le chemin absolu de mon fichier BioTime préalablement télécharger. Il faut que vous changiez le chemin pour travailler avec votre fichier BioTime. 

In [ ]:
#data <- read_csv("C:/Users/marie.parrens/Documents/Enseignement/OptionR_A4/Data/BioTIMEQuery_24_06_2021/BioTIMEQuery_24_06_2021.csv")


Quelle est la dimension des données ? Combien y'a t'il de colonnes ? de lignes ?

In [ ]:
dim(data)

Chaque colonne a un titre. On peut le connaitre grâce à la commande :

In [ ]:
names(data)

On peut obtenir des statistiques "simples" sur chaque colonne grâce à la fonction suivante :

In [ ]:
summary(data)

Regardons les 10 premières lignes du tableau pour se faire une meilleure idée de la structure des données.

In [ ]:
data[1:30,]

Avez vous compris ce que signifie PLOT ? ID_SPECIES ?

## 1) Travail avec un lieu d'étude : sélection des données

Selectionnons, par exemple, les données relative au plot 12.

In [ ]:
data12 <- data[data$PLOT == 12,]

Quelle est la taille du vecteur data12 ? 

In [ ]:
dim(data12)

Le nombre de lignes est très élevé, il y a surement des lignes avec des données maquantes (NA) prises en compte par R. Prenons un petit exemple pour comprendre ce qu'est une valeur manquante. Faisons une enquête auprès de 4 personnes et demandons-leur, leur poids. Les réponses sont : 45, 96 et 78. La quatrième personne n'a pas voulu répondre. Son absence de réponse sera notée NA sous R. 

In [ ]:
poids <- c(45,96,78, NA)
print(poids)

La commande "is.na" nous permet de savoir pour chaque personne, si nous avons la valeur de son poids ou pas.

In [ ]:
is.na(poids)

Pour ne pas fausser les résultats, imaginons que l'on veuille récupérer seulement les données des personnes qui nous ont communiqué leur poids. Pour cela, il suffit de récupérer les indices dont la commande précédente retoure FALSE

In [ ]:
indices <- which(is.na(poids)==FALSE)
print(indices)

C'est bien les 3 premières personnes qui ont donné leur poids. Utilions cette commande pour supprimer les lignes contenants des NA dans data12.

In [ ]:
indices12 <- which(is.na(data12$YEAR)==FALSE)

Combien de données me restent il ?

In [ ]:
length(indices12)

Selectionnons donc dans notre tableau les données chiffrées (celles sans les NA).

In [ ]:
data12ok <- data12[indices12,]
dim(data12ok)

Regardons maintenant le nombre d'espèces differentes présente chaque année pour le PLOT 12

In [ ]:
# nombre d'années où l'on regarder combien d'especes differentes sont présentes.
an <- 1984:2018
EspecesDif <- numeric(length(an))
for (ii in 1:length(an)){
    tmp <- data12ok[data12ok$YEAR == an[ii],]
    taille <- dim(tmp)
    EspecesDif[ii]<-taille[1]
}
print(EspecesDif)

### Représentation graphique des résultats à l'aide de ggplot2

Avant de commencer à faire un graphique avec ggplot2, il faut charger la librairie.

In [ ]:
library(ggplot2)

Graphique montrant l'évolution de la diversité des espèces au cours du temps pour le PLOT 12

In [ ]:
# dataframe contenant les données necessaires à la création du graphique
data.plot <- data.frame(an, EspecesDif)
# Graphique simple
p1<-ggplot(data=data.plot,aes(x=an,y=EspecesDif))+ geom_line(size=1.5,colour="black")+theme_classic()+labs(y="Diversité des espèces", x="Années")
p1

Faisons de même pour un autre PLOT (par exemple, le PLOT 89) et ajoutons la courbe à la figure précédente.

In [ ]:
# PLOT choisi :
n = 89
# selection du PLOT n
dataN <- data[data$PLOT == n,]
# selection des lignes ne contenant pas NA
indicesN <- which(is.na(dataN$YEAR)==FALSE)
dataNok <- dataN[indicesN,]
# nombre d'années où l'on regarder combien d'especes differentes sont présentes.
an <- 1984:2018
EspecesDifN <- numeric(length(an))
for (ii in 1:length(an)){
    tmp <- dataNok[dataNok$YEAR == an[ii],]
    taille <- dim(tmp)
    EspecesDifN[ii]<-taille[1]
}
print(EspecesDifN)
# graphique avec les deux courbes
data.plot2 <- data.frame(an, EspecesDif, EspecesDifN)
p2 <- ggplot(data.plot2,aes(an)) + 
geom_line(aes(y=EspecesDif), size=1.5,colour = "black") + 
geom_line(aes(y=EspecesDifN), size=1.5,colour = "gray")+
labs(x="Années",y="Diversité des espèces")+
expand_limits(y=0)+
theme_classic()

p2

Reprenons le PLOT 12 et représentons l'évolution de chaque espèce en fonction du temps.

In [ ]:
# determination des espèces (ID_SPECIES à considérer)
df <- data12ok
# organisation du tableau en plus petits tableaux ne contenant que la memme espèce
spdf <- split(df, df$ID_SPECIES)
#spdf[1]
# boucle sur chaque petit tableau pour faire la somme de toutes les espèces pendant une meme annnée.
mat <- matrix(data = 0, nrow = length(spdf), ncol = length(an), byrow = FALSE)
for (i in 1:length(an)){
    # chaque petit tableau devient un dataframe
    tmp <- as.data.frame(spdf[i])
    # l'ente de chaque petit tableau est exactement le meme que celle du grand tableau
    colnames(tmp) <- as.character(names(data)) 
    # pour chaque année, on somme l'abondance 
    tmp2 <- tmp %>% group_by(tmp$YEAR) %>% summarise(sum.allrawdata.ABUNDANCE = sum(sum.allrawdata.ABUNDANCE))
    # on ecrit les résultats dans une matrice "nbre espece x annnée"
    for (j in 1:length(tmp2)){
       mat[i,unlist(tmp2[j,1],use.names = FALSE)-an[1]+1] <- unlist(tmp2[j,2], use.names = FALSE)
    }    
}
# parfois, aucune espèce a été détecté sur toute la période -> suppression ligne blanche
somme <- sum(t(mat))
matOK <- mat[somme!=0,]
dim(matOK)
# creation dataframe pour le graphique 
#dataplot <- data.frame(mat)
#colnames(dataplot) <- as.character(an) 
#species <- data12ok %>% group_by(data12ok$ID_SPECIES) %>% summarise(sum.allrawdata.ABUNDANCE = sum(sum.allrawdata.ABUNDANCE))
#rownames(dataplot) <- unlist(species[,1])
#dataplot[450:470,]
#p1<-ggplot(data=dataplot, aes(x=an, y=dataplot[2,]))+ geom_point()+theme_classic()+labs(y="Diversité des espèces", x="Années")


In [ ]:
install.packages("matlab")
library(matlab)
imagesc(matOK, xlab="cols", ylab="rows", col=jet.colors(16))

### 2) Localisation des lieux d'étude

Dans cette partie, on souhaite avoir un aperçu du jeu de données dans son ensemble. Par exemple, on va chercher à localiser les lieux d'étude (PLOT) sur une carte du monde.
La fonction suivante va permettre de dessiner une carte du monde et de représenter un ou des lieux d'études sur la carte.

In [ ]:
drawWorld<-function(lats) {
  world_map<-map_data("world")
	g1<-ggplot()+coord_fixed()+xlab("")+ylab("")
	g1<-g1+geom_polygon(data=world_map, aes(x=long, y=lat, group=group), colour="gray60", fill="gray70")
	g1<-g1+theme(panel.grid.major=element_blank(), panel.grid.minor=element_blank(), 
	panel.background=element_rect(fill="white", colour="white"), axis.line=element_line(colour="white"),
	legend.position="none",axis.ticks=element_blank(), axis.text.x=element_blank(), axis.text.y=element_blank())
	
	if(lats=="y") {
		g1<-g1+geom_hline(yintercept=23.5, colour="red")+geom_hline(yintercept =-23.5, colour="red")
		g1<-g1+geom_hline(yintercept=66.5, colour="darkblue")+geom_hline(yintercept =-66.5, colour="darkblue")
	}
	else { return(g1) }
		return(g1)
}

Cette fonction a un argument. On peut choisir de dessiner les tropiques (arg = "y") ou les lattitudes polaires (arg = "n"). Essatez la fonction avec les deux arguments.

Essyons maintenant de représenter un point sur la carte. Toulouse se situe environ à 43,60°N et 1.44°E. La fonction "geom_point" a besoin d'un dataframe en entrée. Il faut donc écrire les coordonnées de Toulouse sous forme d'un dataframe.

In [ ]:
# creation d'un dataframe contenant les coordonnées de Toulouse. 10 represente la taille du point sur la carte
result <- data.frame(t(c(10,43.60,1.44)))
colnames(result) <- c("res", "lat", "lon")
# graphique : carte du monde + point de Toulouse
carte <- drawWorld("n")+geom_point(data=result, aes(x=lon, y=lat), alpha=I(0.7))
carte

On peut maintenant représenter sur la carte tous les lieux d'étude (PLOT) utilisé dans cette base de données.

In [ ]:
dfplot <- split(data,data$PLOT)
cood <- matrix(data = 0, nrow = length(dfplot), ncol = 2, byrow = TRUE)
for (i in 1 : length(dfplot)){
    tmp2 <- dfplot[i]
    toto <- tmp2[[1]] 
    cood[i,1] <- unlist(toto[1,"LATITUDE"], use.names = FALSE)
    cood[i,2] <- unlist(toto[1,"LONGITUDE"], use.names = FALSE)
}
# preparation du dataframe
plotplot <- data.frame(cood)
col_headings <- c("lat","lon")
names(plotplot) <- col_headings

# Carte + lieux d'étude
carte <- drawWorld("n")+geom_point(data=plotplot, aes(x=lon, y=lat), alpha=I(0.7))
carte